# 06 — Tableau Preparation  

In this notebook, we transform our cleaned Olist datasets into **analytics-ready extracts** for Tableau.  
Our goal is to:  
- Create **dimension tables** (customers, sellers, products, calendar)  
- Create **fact tables** (orders, order items)  
- Generate **pre-aggregated KPI extracts** (monthly KPIs, customer cohorts)  
- Export everything to `data/processed/` for direct Tableau use  


## Step 1 — Load Cleaned Data  

We start by loading the cleaned datasets from `data/processed/`. These are the outputs of our data cleaning and feature engineering steps.  


In [9]:
import pandas as pd
import os

base_path = "../data/processed/"

customers = pd.read_csv(os.path.join(base_path, "customers_cleaned.csv"))
orders = pd.read_csv(os.path.join(base_path, "orders_cleaned.csv"))
order_items = pd.read_csv(os.path.join(base_path, "order_items_cleaned.csv"))
payments = pd.read_csv(os.path.join(base_path, "payments_cleaned.csv"))
products = pd.read_csv(os.path.join(base_path, "products_cleaned.csv"))
sellers = pd.read_csv(os.path.join(base_path, "sellers_cleaned.csv"))
reviews = pd.read_csv(os.path.join(base_path, "reviews_cleaned.csv"))
categories = pd.read_csv(os.path.join(base_path, "categories_cleaned.csv"))


## Step 2 — Normalize Dates  

Since Tableau relies heavily on time-series data, we ensure all date columns are properly converted to `datetime`.  


In [10]:
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    if col in orders.columns:
        orders[col] = pd.to_datetime(orders[col], errors="coerce")


## Step 3 — Build Dimension Tables  

Dimension tables act as lookups for Tableau. They allow us to slice metrics by geography, product categories, or time.  

- **Customers** → ID, city, state  
- **Sellers** → ID, city, state  
- **Products** → merged with category translations  
- **Calendar** → a continuous daily calendar for flexible time-series analysis  


In [11]:
dim_customers = customers[["customer_id", "customer_unique_id", "customer_city", "customer_state"]]
dim_sellers = sellers[["seller_id", "seller_city", "seller_state"]]

dim_products = products.merge(categories, on="product_category_name", how="left")

calendar = pd.DataFrame({
    "date": pd.date_range(orders["order_purchase_timestamp"].min(),
                          orders["order_purchase_timestamp"].max())
})
calendar["year"] = calendar["date"].dt.year
calendar["month"] = calendar["date"].dt.month
calendar["year_month"] = calendar["date"].dt.to_period("M")


## Step 4 — Fact Table: Order Items  

The **order items fact table** contains one row per product sold, enriched with product and seller details.  


In [12]:
fact_order_items = order_items.merge(
    orders[["order_id", "customer_id", "order_purchase_timestamp"]],
    on="order_id", how="left"
)
fact_order_items = fact_order_items.merge(
    dim_products[["product_id", "product_category_name_english"]],
    on="product_id", how="left"
)
fact_order_items = fact_order_items.merge(dim_sellers, on="seller_id", how="left")


## Step 5 — Fact Table: Orders  

The **orders fact table** summarizes revenue, payments, and customer satisfaction at the order level.  


In [13]:
order_payments = payments.groupby("order_id", as_index=False)["payment_value"].sum()
order_reviews = reviews.groupby("order_id", as_index=False)["review_score"].mean()

fact_orders = orders.merge(order_payments, on="order_id", how="left")
fact_orders = fact_orders.merge(order_reviews, on="order_id", how="left")


## Step 6 — Monthly KPI Extract  

To speed up Tableau dashboards, we pre-aggregate **monthly KPIs**:  
- Orders count  
- Revenue  
- Average review score  


In [14]:
monthly_kpis = fact_orders.copy()
monthly_kpis["year_month"] = monthly_kpis["order_purchase_timestamp"].dt.to_period("M")

monthly_kpis = monthly_kpis.groupby("year_month").agg({
    "order_id": "count",
    "payment_value": "sum",
    "review_score": "mean"
}).reset_index()

monthly_kpis.rename(columns={
    "order_id": "order_count",
    "payment_value": "total_revenue",
    "review_score": "avg_review_score"
}, inplace=True)


## Step 7 — Customer Cohorts  

We track **customer retention** by grouping customers into cohorts based on their first purchase month.  


In [15]:
orders["cohort_month"] = orders.groupby("customer_id")["order_purchase_timestamp"].transform("min").dt.to_period("M")
orders["order_month"] = orders["order_purchase_timestamp"].dt.to_period("M")

cohort_data = orders.groupby(["cohort_month", "order_month"])["customer_id"].nunique().reset_index()
cohort_data.rename(columns={"customer_id": "active_customers"}, inplace=True)


## Step 8 — Save Extracts  

Finally, we export all extracts into `data/processed/`. These are **direct Tableau inputs**.  


In [16]:
output_path = "../data/processed/"

dim_customers.to_csv(os.path.join(output_path, "dim_customers.csv"), index=False)
dim_sellers.to_csv(os.path.join(output_path, "dim_sellers.csv"), index=False)
dim_products.to_csv(os.path.join(output_path, "dim_products.csv"), index=False)
calendar.to_csv(os.path.join(output_path, "dim_calendar.csv"), index=False)

fact_order_items.to_csv(os.path.join(output_path, "fact_order_items.csv"), index=False)
fact_orders.to_csv(os.path.join(output_path, "fact_orders.csv"), index=False)

monthly_kpis.to_csv(os.path.join(output_path, "monthly_kpis.csv"), index=False)
cohort_data.to_csv(os.path.join(output_path, "customer_cohorts.csv"), index=False)


## Wrap-Up  

✅ In this notebook, we created Tableau-ready extracts:  
- **Dimensions**: Customers, Sellers, Products, Calendar  
- **Facts**: Orders, Order Items  
- **KPI Extracts**: Monthly KPIs, Customer Cohorts  

These CSVs can now be directly connected to Tableau dashboards.  


In [1]:
import pandas as pd

# Load engineered feature sets from interim
customer_final = pd.read_csv("../data/interim/customer_features.csv")
seller_final   = pd.read_csv("../data/interim/seller_features.csv")
product_final  = pd.read_csv("../data/interim/product_features.csv")
order_final    = pd.read_csv("../data/interim/order_features.csv")
payment_final  = pd.read_csv("../data/interim/payment_features.csv")

print("✅ Interim features loaded successfully")


✅ Interim features loaded successfully


In [2]:
# Save feature sets to processed folder for Tableau
customer_final.to_csv("../data/processed/customer_features.csv", index=False)
seller_final.to_csv("../data/processed/seller_features.csv", index=False)
product_final.to_csv("../data/processed/product_features.csv", index=False)
order_final.to_csv("../data/processed/order_features.csv", index=False)
payment_final.to_csv("../data/processed/payment_features.csv", index=False)

print("✅ Features exported to data/processed/ for Tableau")


✅ Features exported to data/processed/ for Tableau


In [3]:
import os

# Define processed path
processed_path = "../data/processed/"

# List all CSV files in processed/
files = [f for f in os.listdir(processed_path) if f.endswith(".csv")]

print("✅ Processed files available for Tableau:")
for f in files:
    df = pd.read_csv(os.path.join(processed_path, f))
    print(f" - {f}: {df.shape[0]} rows, {df.shape[1]} columns")


✅ Processed files available for Tableau:
 - categories_cleaned.csv: 71 rows, 2 columns
 - customers_cleaned.csv: 99441 rows, 5 columns
 - customer_cohorts.csv: 25 rows, 3 columns
 - customer_features.csv: 99441 rows, 9 columns
 - dim_calendar.csv: 773 rows, 4 columns
 - dim_customers.csv: 99441 rows, 4 columns
 - dim_products.csv: 32951 rows, 10 columns
 - dim_sellers.csv: 3095 rows, 3 columns
 - fact_orders.csv: 99441 rows, 10 columns
 - fact_order_items.csv: 112650 rows, 12 columns
 - geolocation_cleaned.csv: 738332 rows, 5 columns
 - monthly_kpis.csv: 25 rows, 4 columns
 - orders_cleaned.csv: 99441 rows, 8 columns
 - order_features.csv: 99441 rows, 14 columns
 - order_items_cleaned.csv: 112650 rows, 7 columns
 - payments_cleaned.csv: 103886 rows, 5 columns
 - payment_features.csv: 103886 rows, 6 columns
 - products_cleaned.csv: 32951 rows, 9 columns
 - product_features.csv: 32951 rows, 6 columns
 - reviews_cleaned.csv: 99224 rows, 7 columns
 - sellers_cleaned.csv: 3095 rows, 4 colum